# Day 16 - Packet Decoder

https://adventofcode.com/2021/day/16

In [30]:
from pathlib import Path

INPUTS = Path("input.txt").read_text().strip()


For this challenge, the trick won't really be in evaluating the hex format, but in parsing the bits version. So, let's take care of converting it straight away.

In [31]:
hex_to_bits = {
    "0": "0000",
    "1": "0001",
    "2": "0010",
    "3": "0011",
    "4": "0100",
    "5": "0101",
    "6": "0110",
    "7": "0111",
    "8": "1000",
    "9": "1001",
    "A": "1010",
    "B": "1011",
    "C": "1100",
    "D": "1101",
    "E": "1110",
    "F": "1111",
}
BITS = "".join(hex_to_bits[x] for x in INPUTS)


## Part 1

For this one I'm taking a functional approach, writing individual functions capable of pulling necessary details out of the "bit stream" as I'm calling it. The format described in the challenge lends itself to parsing only pieces of the transmission at a time, then acting on the `remaining` bits at a later point.

We can't always be certain how big a given packet or sub-packet is going to be, hence we just keep chopping off what we can parse now and dealing with those remainders later. To that end, each of the functions below returns some tuple, with the last portion always being `remaining` for the bits that have not been consumed.

In [32]:
def version_id(bit_stream: str) -> tuple[dict[str, str | int], str]:
    """Splits `bit_stream` into a tuple of `(version, id, remaining)`.

    The `version` and `id` are consumed from the incoming bit stream and converted
    to their decimal values. All `remaining` parts are returned unchanged.
    """
    version, type_id, remaining = bit_stream[:3], bit_stream[3:6], bit_stream[6:]
    return int(version, base=2), int(type_id, base=2), remaining


def literal_value(bit_stream: str) -> tuple[int, str]:
    """Returns a literal value from a packet.

    The packet should already have its version and ID split off
    from a prior operation.
    """
    value = ""
    remaining = bit_stream
    while True:
        section, remaining = remaining[:5], remaining[5:]
        value += section[1:]
        if section[0] == "0":
            break
    return int(value, base=2), remaining


def operator_packet(bit_stream: str) -> tuple[dict[str, str | int], str]:
    """Parses the bit stream to return tuple `(data, remaining)`.

    `data` contains either key `sub_bit_length`, the length (in bits) of sub-packets;
    or `num_packets`, the number of sub-packets that follow.

    `remaining` contains the remaining portions of the bit_stream that were
    not processed.
    """
    length, remaining = bit_stream[0], bit_stream[1:]
    data = {}
    if length == "0":
        # 15 bits represent length in bits of the sub-packets
        sub_bit_length, remaining = remaining[:15], remaining[15:]
        data["sub_bit_length"] = int(sub_bit_length, base=2)
    else:
        # 11 bits represent number of sub-packets
        num_packets, remaining = remaining[:11], remaining[11:]
        data["num_packets"] = int(num_packets, base=2)
    return data, remaining


Now we can get down to the business of parsing the data. Because we're dealing with sub-packets and potentially sub-sub-packets, we need to think recursively, handling as much as we can, pulling whole packet sections out to deal with at a time, and passing the remaining bits back up to be dealt with later (similar to the parsing functions used above).

In [33]:
from typing import Any


def get_packet_data(bit_stream: str) -> dict[str, Any]:
    if len(set(bit_stream)) < 2:
        return None, None
    version, type_id, remaining = version_id(bit_stream=bit_stream)
    packet = {
        "version": version,
        "type_id": type_id,
    }
    if type_id == 4:
        packet["data"], remaining = literal_value(remaining)
    else:
        data, remaining = operator_packet(remaining)
        packet["sub_packets"] = []
        if "sub_bit_length" in data:
            # length type 0 got us here. We need to chop off a chunk of the remaining
            # into the set of "packets" we need to work on, then consume them individually
            # until the entire stream is empty (ending in a few leftover 0 bits, that is)
            length = data["sub_bit_length"]
            packets, remaining = remaining[:length], remaining[length:]
            while packets is not None and set(packets) != {"0"}:
                sub_packet, packets = get_packet_data(bit_stream=packets)
                if sub_packet is not None:
                    packet["sub_packets"].append(sub_packet)
        elif "num_packets" in data:
            # length type 1, indicating a number of packets of arbitrary length.
            # We just need to iterate a number of times equal to num packets
            # and process individual packets recursively.
            for _ in range(data["num_packets"]):
                sub_packet, remaining = get_packet_data(bit_stream=remaining)
                if sub_packet is not None:
                    packet["sub_packets"].append(sub_packet)
    return packet, remaining


# Get our packet data and ignore "remaining", which will be None anyway.
PACKETS, _ = get_packet_data(BITS)


Now `PACKETS` contains the completely-parsed data from the bit stream into a Python object, namely a dict which may contain lists of other dicts of the same basic structure.

(Sure, I could have written a class to contain this data, but I got lazy.)

The actual challenge of Part 1 is to sum all the versions from all packets and sub-packets. For that we need to recursively go over our `PACKETS` and pull up the `version` keys at every level.

In [34]:
def get_versions(packet: dict[str, Any]) -> int:
    version = packet["version"]
    sub_packets = packet.get("sub_packets", [])
    version += sum([get_versions(packet=x) for x in sub_packets])
    return version


That brings us to our solution:

In [35]:
version_sum = get_versions(PACKETS)
print(f"Sum of versions: {version_sum}")

Sum of versions: 889


## Part 2

For reference, here's the AoC site's description of the type IDs and their meanings (reformatted here for clarity):

> Literal values (type ID `4`) represent a single number as described above. The remaining type IDs are more interesting:

> - Packets with type ID `0` are **sum** packets - their value is the sum of the values of their sub-packets. If they only have a single sub-packet, their value is the value of the sub-packet.
> - Packets with type ID `1` are **product** packets - their value is the result of multiplying together the values of their sub-packets. If they only have a single sub-packet, their value is the value of the sub-packet.
> - Packets with type ID `2` are **minimum** packets - their value is the minimum of the values of their sub-packets.
> - Packets with type ID `3` are **maximum** packets - their value is the maximum of the values of their sub-packets.
> - Packets with type ID `5` are **greater than** packets - their value is `1` if the value of the first sub-packet is greater than the value of the second sub-packet; otherwise, their value is `0`. These packets always have exactly two sub-packets.
> - Packets with type ID `6` are **less than** packets - their value is `1` if the value of the first sub-packet is less than the value of the second sub-packet; otherwise, their value is `0`. These packets always have exactly two sub-packets.
> - Packets with type ID `7` are **equal to** packets - their value is `1` if the value of the first sub-packet is equal to the value of the second sub-packet; otherwise, their value is `0`. These packets always have exactly two sub-packets.

So, all we need to do is process our already-decoded packet dicts to evaluate the final value. We'll still need a recursive solution, of course, due to the deeply-nested nature of these packets, but that shouldn't be a problem.

My approach will be just running a match-case on the `type_id` for a packet, then taking different actions based on those outcomes. Certainly a good excuse to flex the new syntax in Python 3.10. 😊

In [36]:
def evaluate_packet(packet: dict[str, Any]) -> int:
    sub_packets = packet.get('sub_packets', [])
    if packet['type_id'] in (5, 6, 7):
        # To make things a little more DRY, we'll evaluate these ones
        # early in these cases alone, as we can be sure that only two sub-packets
        # exist within these types.
        comparisons = [
            evaluate_packet(packet=sub_packets[0]),
            evaluate_packet(packet=sub_packets[1]),
        ]
    match packet['type_id']:
        case 0:
            # Sum
            return sum([evaluate_packet(packet=x) for x in sub_packets])
        case 1:
            # Product
            product = 1
            for sub_packet in sub_packets:
                product *= evaluate_packet(packet=sub_packet)
            return product
        case 2:
            # Minimum
            return min([evaluate_packet(packet=x) for x in sub_packets])
        case 3:
            # Maximum
            return max([evaluate_packet(packet=x) for x in sub_packets])
        case 4:
            # Literal value: return our own 'data' key
            return packet['data']
        case 5:
            # Greater than
            # NOTE: int(True) == 1, int(False) == 0.
            return int(comparisons[0] > comparisons[1])
        case 6:
            # Less than
            return int(comparisons[0] < comparisons[1])
        case 7:
            # Equal to
            return int(comparisons[0] == comparisons[1])

That all settled, we just need to run it and get our result:

In [37]:
final_value = evaluate_packet(packet=PACKETS)
print(f"Value of this transmission: {final_value}")

Value of this transmission: 739303923668
